# Infrastructure as Code Testing
In this notebook we will use the IaC ideas to see how we can manipulate the Redshift cluster and pull from an S3 bucket.

Import libraries.

In [ ]:
import pandas as pd
import psycopg2
import boto3
import configparser

Read in the configuration file.

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

In [ ]:
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3', region_name='eu-west-2',
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET)

iam = boto3.client('iam', region_name='eu-west-2',
                   aws_access_key_id=KEY,
                   aws_secret_access_key=SECRET)

redshift = boto3.client('redshift', region_name='eu-west-2',
                        aws_access_key_id=KEY,
                        aws_secret_access_key=SECRET)

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier='redshift-cluster-1')['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

## Connect to the Redshift cluster

The code below loads an SQL extension and then connects to the Amazon Redshift cluster with the properties as defined in the config file.

In [ ]:
%load_ext sql

In [ ]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

## Drop existing tables

In [ ]:
from sql_queries import drop_table_queries

for query in drop_table_queries:
    cur.execute(query)
    conn.commit()

## Create the tables in the cluster

In [ ]:
from sql_queries import create_table_queries

for query in create_table_queries:
    cur.execute(query)
    conn.commit()

## COPY data into the staging tables

In [ ]:
from sql_queries import copy_table_queries

for query in copy_table_queries:
    cur.execute(query)
    conn.commit()

## INSERT data into final tables

In [ ]:
from sql_queries import insert_table_queries

for query in insert_table_queries:
    cur.execute(query)
    conn.commit()

## Tidy up the resources

In [ ]:
#redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)